<a href="https://colab.research.google.com/github/marco-siino/llm-network-testbed/blob/main/blueprint/simulated-environment/codeLlama/1-shot-prompt/CodeLlama_7B_IT_BluePrintToNS3_1_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and load the model.

In [1]:
import os
import random
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

model_id = 'cache/models--meta-llama--CodeLlama-7b-Instruct-hf/snapshots/4ce0c40b2ea823bd1d8f1f3fd5bc8a7e80d749bc'

#model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

# I can decide which GPU to use on this node on Leonardo.
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=dtype,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
!nvidia-smi

Wed Dec  4 14:57:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB            On | 00000000:1D:00.0 Off |                    0 |
| N/A   44C    P0               75W / 459W|   3431MiB / 65536MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Generate the NS3 code for the smart_agriculture scenario.

In [3]:
with open("projects/testbed-at-JNCA/blueprint/sample_blueprint.json") as f:
        sample_blueprint = f.read()
with open("projects/testbed-at-JNCA/blueprint/sample_ns3_code.cc") as f:
        sample_ns3_code = f.read()
with open("projects/testbed-at-JNCA/blueprint/smart_agriculture.json") as f:
        test_blueprint = f.read()

In [8]:
prompt = """
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        """+sample_blueprint+ """

        SAMPLE NS3 CODE:

        """+sample_ns3_code+"""

        TEST BLUEPRINT:

        """+test_blueprint+"""

        TEST NS3 CODE:

        """
print(prompt)


 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "components": [

    {"type": "WiFi AP",
      "id": "AP-001",
      "application": "WiFi Access Point",
      "location": "City Hall",      
      "protocol": "WiFi",
      "ip_address":"192.168.1.1",        
      "latitude": 38.10351066811091,
      "longitude": 13.3459399220741
    },
    {
      "type": "Smart Thermostat",
      "id": "ST-001",
      "location": "Hallway",
      "temperature_range": "18-25°C",
      "energy_efficiency": "A+" ,     
      "protocol":"Wi-Fi",
      "latitude": 38.10863528672426,
      "longitude": 13.34050633101243,
      "ip_address":"192.168.1.2"
    },
    {
      "type": "Smart Light Bulb",
      "id": "SLB-001",
      "location": "Kitchen",
      "b

In [17]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=5500)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s> 
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "components": [

    {"type": "WiFi AP",
      "id": "AP-001",
      "application": "WiFi Access Point",
      "location": "City Hall",      
      "protocol": "WiFi",
      "ip_address":"192.168.1.1",        
      "latitude": 38.10351066811091,
      "longitude": 13.3459399220741
    },
    {
      "type": "Smart Thermostat",
      "id": "ST-001",
      "location": "Hallway",
      "temperature_range": "18-25°C",
      "energy_efficiency": "A+" ,     
      "protocol":"Wi-Fi",
      "latitude": 38.10863528672426,
      "longitude": 13.34050633101243,
      "ip_address":"192.168.1.2"
    },
    {
      "type": "Smart Light Bulb",
      "id": "SLB-001",
      "location": "Kitchen",
    

## Save the .cc code into a file.

In [25]:
s = tokenizer.decode(outputs[0])
s = s.split("TEST NS3 CODE:", 1)[-1].lstrip().rstrip()

with open("projects/testbed-at-JNCA/blueprint/codellama/smart_agriculture-1_shot.cc", "w") as file:
    file.write(s)

# Generate the NS3 code for the smart_city scenario.

In [29]:
with open("projects/testbed-at-JNCA/blueprint/smart_city.json") as f:
        test_blueprint = f.read()

In [33]:
prompt = """
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        """+sample_blueprint+ """

        SAMPLE NS3 CODE:

        """+sample_ns3_code+"""

        TEST BLUEPRINT:

        """+test_blueprint+"""

        TEST NS3 CODE:

        """
print(prompt)


 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "components": [

    {"type": "WiFi AP",
      "id": "AP-001",
      "application": "WiFi Access Point",
      "location": "City Hall",      
      "protocol": "WiFi",
      "ip_address":"192.168.1.1",        
      "latitude": 38.10351066811091,
      "longitude": 13.3459399220741
    },
    {
      "type": "Smart Thermostat",
      "id": "ST-001",
      "location": "Hallway",
      "temperature_range": "18-25°C",
      "energy_efficiency": "A+" ,     
      "protocol":"Wi-Fi",
      "latitude": 38.10863528672426,
      "longitude": 13.34050633101243,
      "ip_address":"192.168.1.2"
    },
    {
      "type": "Smart Light Bulb",
      "id": "SLB-001",
      "location": "Kitchen",
      "b

In [34]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=5500)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s> 
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "components": [

    {"type": "WiFi AP",
      "id": "AP-001",
      "application": "WiFi Access Point",
      "location": "City Hall",      
      "protocol": "WiFi",
      "ip_address":"192.168.1.1",        
      "latitude": 38.10351066811091,
      "longitude": 13.3459399220741
    },
    {
      "type": "Smart Thermostat",
      "id": "ST-001",
      "location": "Hallway",
      "temperature_range": "18-25°C",
      "energy_efficiency": "A+" ,     
      "protocol":"Wi-Fi",
      "latitude": 38.10863528672426,
      "longitude": 13.34050633101243,
      "ip_address":"192.168.1.2"
    },
    {
      "type": "Smart Light Bulb",
      "id": "SLB-001",
      "location": "Kitchen",
    

## Save the .cc code into a file.

In [35]:
s = tokenizer.decode(outputs[0])
s = s.split("TEST NS3 CODE:", 1)[-1].lstrip().rstrip()

with open("projects/testbed-at-JNCA/blueprint/codellama/smart_city-1_shot.cc", "w") as file:
    file.write(s)

# Generate the NS3 code for the smart_home scenario.

In [36]:
with open("projects/testbed-at-JNCA/blueprint/smart_home.json") as f:
        test_blueprint = f.read()

In [37]:
prompt = """
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        """+sample_blueprint+ """

        SAMPLE NS3 CODE:

        """+sample_ns3_code+"""

        TEST BLUEPRINT:

        """+test_blueprint+"""

        TEST NS3 CODE:

        """
print(prompt)


 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "components": [

    {"type": "WiFi AP",
      "id": "AP-001",
      "application": "WiFi Access Point",
      "location": "City Hall",      
      "protocol": "WiFi",
      "ip_address":"192.168.1.1",        
      "latitude": 38.10351066811091,
      "longitude": 13.3459399220741
    },
    {
      "type": "Smart Thermostat",
      "id": "ST-001",
      "location": "Hallway",
      "temperature_range": "18-25°C",
      "energy_efficiency": "A+" ,     
      "protocol":"Wi-Fi",
      "latitude": 38.10863528672426,
      "longitude": 13.34050633101243,
      "ip_address":"192.168.1.2"
    },
    {
      "type": "Smart Light Bulb",
      "id": "SLB-001",
      "location": "Kitchen",
      "b

In [39]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=5500)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s> 
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "components": [

    {"type": "WiFi AP",
      "id": "AP-001",
      "application": "WiFi Access Point",
      "location": "City Hall",      
      "protocol": "WiFi",
      "ip_address":"192.168.1.1",        
      "latitude": 38.10351066811091,
      "longitude": 13.3459399220741
    },
    {
      "type": "Smart Thermostat",
      "id": "ST-001",
      "location": "Hallway",
      "temperature_range": "18-25°C",
      "energy_efficiency": "A+" ,     
      "protocol":"Wi-Fi",
      "latitude": 38.10863528672426,
      "longitude": 13.34050633101243,
      "ip_address":"192.168.1.2"
    },
    {
      "type": "Smart Light Bulb",
      "id": "SLB-001",
      "location": "Kitchen",
    

## Save the .cc code into a file.

In [40]:
s = tokenizer.decode(outputs[0])
s = s.split("TEST NS3 CODE:", 1)[-1].lstrip().rstrip()

with open("projects/testbed-at-JNCA/blueprint/codellama/smart_home-1_shot.cc", "w") as file:
    file.write(s)